# Recommendation systems
Welcome to your first homework assignment about recommendation systems.


## The Cold Start Problem 

The colaborative filtering method discussed in class does not address the problem of new user or new movies. What prediction would you use in these cases:

* A new user but a known movie
* A new movie and a known user
* A new user and new movie

    1.For a new user, we should ask the user to select some movies he likes, so that we can find the similarity between this user and the other users, and then predict whether this user's preference.  
   
    2.For a new movie, we should look at its features, such as the categories, directors and actors, and find the similarity bwtween it and other movies. Then based on the watching or rating history of a user, we can predict whether he will like this movie.  

    3.For a new user, we should ask the user to rate some movies or select some movies she or he likes, so that we can find the similarity between this user and the other users. For the new movie, we should look at its features such as the direcctors, actors and find the similarity bwtween it and other movies. So based on the preference of similar users, and whether the movie is similar to the preference movies of those users, we can predict whether this new user will like this new movie.

## Matrix Factorization with bias
We want to extend the Matrix Factorization model discussed in class to add a "bias" parameter for each user and another "bias" parameter for each movie.  For the problem in class we had the parameters matrix $U$ and $V$, we are adding $u^0$ which is a vector of dimension $n_u$ and $v^0$ which is a vector of dimension $n_m$. The equations

$$\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $$ 
 
(a) How many weights (parameters) are we fitting for this problem?

(b) Write the gradient descent equations for this problem.

(a)  
$$ (n_u + n_m)*k + n_u + n_m $$
(b)  
$$ u_{0i} = u_{0i}+\frac{2\eta}{N}\sum_{i,j: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}  $$
$$ v_{0j} = v_{0i}+\frac{2\eta}{N}\sum_{i,j: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}  $$
$$ u_{ik} = u_i+\frac{2\eta}{N}\sum_{j: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}v_{jk}  $$
$$ v_{jk} = v_j+\frac{2\eta}{N}\sum_{i: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}u_{ik}  $$

# Collaborative Filtering with Gradient Descent 

In this part of the assignment you will build a collaborative filtering model to predict netflix ratings.  This assignment will step you through how to do this using stochastic gradient descent. The data for this assignment is ...

**Instructions:**
- Do not use loops (for/while) in your code, unless the instructions explicitly ask you to do so.
- DO NOT change paths (-3 points)
- DO NOT submit data to github (-2 points)

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Encoding rating data
    - Initializing parameters
    - Calculating the cost function
    - Calculating gradient
    - Using an optimization algorithm (gradient descent) 
    - Predicting on new data
- Putting it all together.

In [1]:
import numpy as np
import pandas as pd

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [2]:
# The first row says that user 1 reated movie 11 with a score of 4
!cat tiny_training2.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [3]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [4]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    # YOUR CODE HERE
    try:
        user_to_index, df['userId'], num_users = proc_col(df['userId'])
        movie_to_index, df['movieId'], num_movies = proc_col(df['movieId'])
    except:
        raise NotImplementedError()
    return df, num_users, num_movies

In [5]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)

In [6]:
assert(num_users == 7)

In [7]:
assert(num_movies == 4)

In [8]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

## Initializing parameters

In [9]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[ 3.55790894,  4.69774849,  0.92361109,  1.58739544,  3.00593239],
       [ 4.69774849,  7.44656163,  1.18135616,  2.64524868,  4.74559066],
       [ 0.92361109,  1.18135616,  0.24548062,  0.34025121,  0.69616965],
       [ 1.58739544,  2.64524868,  0.34025121,  1.61561   ,  2.41361975],
       [ 3.00593239,  4.74559066,  0.69616965,  2.41361975,  3.82505541],
       [ 2.02000808,  3.29656257,  0.43174569,  2.065911  ,  3.07264619],
       [ 2.07691001,  3.02887291,  0.53270924,  1.02482544,  1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [10]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [17]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [18]:
print(Y[3])

  (0, 0)	5
  (0, 3)	2


#### def sparse_multiply(df, emb_user, emb_movie):
    """ This function returns U*V^T element wise multi by R as a sparse matrix.
    
    It avoids creating the dense matrix U*V^T
    """
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

In [20]:
def sparse_multiply(df, emb_user, emb_movie):
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values],axis=1)
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0],column_name='Prediction')

## Calculating the cost function

In [21]:
# Use vectorized computation for this function. No loops!
# Hint: use df2matrix and sparse_multiply
def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: e
      mbedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    # YOUR CODE HERE
    try: 
        sparse_prediction_matrix = sparse_multiply(df, emb_user, emb_movie)
        Y = df2matrix(df, num_users, num_movies)
        num_nonzero = Y.count_nonzero()
        error = np.sum((Y-sparse_prediction_matrix).power(2))/num_nonzero
    except:
        raise NotImplementedError()
    return error

In [22]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

## Calculating gradient

In [23]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [24]:
def gradient(df, Y, emb_user, emb_movie):
    """ Computes the gradient.
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    # YOUR CODE HERE
    try:
        sparse_prediction_matrix = sparse_multiply(df, emb_user, emb_movie)
        num_nonzero = Y.count_nonzero()
        d_emb_user = -2/num_nonzero*(Y - sparse_prediction_matrix)*emb_movie
        d_emb_movie = -2/num_nonzero*(Y - sparse_prediction_matrix).transpose()*emb_user
    except:
        raise NotImplementedError()
    return d_emb_user, d_emb_movie

In [25]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)

In [38]:
grad_user

array([[-0.12243115, -0.17937243, -0.12292687],
       [ 0.31426962,  0.55623039,  0.6459187 ],
       [-0.96440297, -1.44475001, -1.05269757],
       [-0.63049179, -0.74711157, -0.35949076],
       [ 0.12479118,  0.42249281, -0.04343876],
       [-0.39796164, -0.02697214, -0.41242749],
       [ 0.0509932 ,  0.53928159,  0.28188905]])

In [26]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [27]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Using gradient descent with momentum

In [28]:
# you can use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    # YOUR CODE HERE
    try:
        sparse_prediction_matrix = sparse_multiply(df, emb_user, emb_movie)
        num_nonzero = Y.count_nonzero()
        # initialization
        d_emb_user = -2/num_nonzero*(Y - sparse_prediction_matrix)*emb_movie
        d_emb_movie = -2/num_nonzero*(Y - sparse_prediction_matrix).transpose()*emb_user
        for i in range(iterations):
            sparse_prediction_matrix = sparse_multiply(df, emb_user, emb_movie)
            d_emb_user = 0.9*d_emb_user + 0.1*(-2/num_nonzero*(Y - sparse_prediction_matrix)*emb_movie)
            d_emb_movie = 0.9*d_emb_movie + 0.1*(-2/num_nonzero*(Y - sparse_prediction_matrix).transpose()*emb_user)          
            emb_user = emb_user - learning_rate * d_emb_user
            emb_movie = emb_movie - learning_rate * d_emb_movie
            if i%50 == 0:
                print('training cost:', cost(df, emb_user, emb_movie))
                if isinstance(df_val, pd.DataFrame):
                    print('validation cost:', cost(df_val, emb_user, emb_movie))
    except:
        raise NotImplementedError()
    return emb_user, emb_movie

In [29]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

training cost: 4.65294604223
training cost: 1.67550441207
training cost: 0.967134455421
training cost: 0.695089266907


In [30]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.53)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [31]:
def proc_new_col(col_val, col_train):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col_train.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    col_train = np.array([name2idx[x] for x in col_train])
    col_val = np.array([name2idx[x] if (x in name2idx) else -1 for x in col_val ])
    return col_val, col_train

In [32]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    # YOUR CODE HERE
    try:
        df_val['userId'], df_train['userId'] = proc_new_col(df_val['userId'], df_train['userId'])
        df_val['movieId'],df_train['movieId'] = proc_new_col(df_val['movieId'], df_train['movieId'])      
        df_val = df_val[(df_val['userId'] != -1) & (df_val['movieId'] != -1)]
    except:
        raise NotImplementedError()
    return df_val

In [33]:
df_t = pd.read_csv("tiny_training2.csv")
df_v = pd.read_csv("tiny_val2.csv")
df_v = encode_new_data(df_v, df_t)

In [34]:
assert(len(df_v.userId.unique())==2)

In [35]:
assert(len(df_v) == 2)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [36]:
# Don't change this path use a simlink if you have the data somewhere else
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20205 19507


In [37]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000, learning_rate=1, df_val=df_val)

training cost: 12.383564165
validation cost: 12.4905394575
training cost: 9.93343975583
validation cost: 10.0706045005
training cost: 7.17519577398
validation cost: 7.31107269652
training cost: 5.1969019147
validation cost: 5.32759735889
training cost: 4.05886669569
validation cost: 4.1833374462
training cost: 3.32972590874
validation cost: 3.442954854
training cost: 2.82408451386
validation cost: 2.92735317655
training cost: 2.45762646316
validation cost: 2.55412442201
training cost: 2.18277822953
validation cost: 2.27508589381
training cost: 1.97066105087
validation cost: 2.06056007952
training cost: 1.80292238279
validation cost: 1.89162472796
training cost: 1.66747572603
validation cost: 1.75581972772
training cost: 1.55612199434
validation cost: 1.64470004418
training cost: 1.46315114114
validation cost: 1.55239027519
training cost: 1.38448880296
validation cost: 1.47470241892
training cost: 1.31716118514
validation cost: 1.40858302885
training cost: 1.25895194093
validation cost:

In [39]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

0.766573002646 0.910427187685


In [40]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.77)

In [41]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=2) == 0.91)

#  Advanced Recommendation (Optional)
Use Regression / Random forest to add other features to your recommendation algorithm. Here are the steps:

* Here are potential features: 
    * user embeding, movie embeding (from the hw)
    * movie genres
    * daypart features
    
Other extentions: add regularization to gradient descent

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()